<a href="https://colab.research.google.com/github/Sarvesh1814/US-Book-Recommendation-System-/blob/main/ReRanking_XGB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training Reranker


## Importing Libraries

In [ ]:
import numpy as np
from sklearn.datasets import load_svmlight_file
from sklearn.model_selection import train_test_split
import pandas as pd
import pickle
import scipy

## Loading Dataset

In [ ]:
Base = "/content/drive/MyDrive/RS Data/Assignment 3/Processed_data/"
train=pd.read_csv(Base+"Train_df_final.csv")
test = pd.read_csv(Base+"Test_df_final.csv")
train = train[["User-ID"	,"ISBN",	"Book-Rating"	,"book_name"]]
test = test[["User-ID"	,"ISBN",	"Book-Rating"	,"book_name"]]

In [ ]:
datas = pd.read_csv("/content/drive/MyDrive/RS Data/Assignment 3/Processed_data/Books_with_108_features.csv")

In [ ]:
users= pd.read_csv("/content/drive/MyDrive/RS Data/Assignment 3/Processed_data/users.csv")

## Train Features

In [ ]:
# Feteching the features of Books and Users and merging them to make common feature table

train_features = train.merge(datas,on="ISBN",how="left")
train_features = train_features.merge(users,on="User-ID",how="left")

In [ ]:
train_features

,User-ID,ISBN,Book-Rating,book_name,Book-Title,page_count,categories,Synopsis,Book-Author,Year-Of-Publication,...,feature_95,feature_96,feature_97,feature_98,feature_99,feature_100,Unnamed: 0,Location,Age,country
0,68436,0345323440,8,Footfall,Footfall,706.0,['Fiction'],hostile extraterrestrial known fithp invade ea...,Larry Niven,1996,...,0.468369,0.346564,0.053775,-0.575963,0.202055,0.013086,68435,"downieville, california, usa",49.0,usa
1,216829,0375707972,5,The Reader,The Reader,224.0,['Fiction'],hailed coiled eroticism moral claim make upon ...,Bernhard Schlink,1999,...,0.671669,0.287058,0.089154,-0.696850,-0.135784,0.029446,216828,"parker, colorado, usa",NaN,usa
2,153718,0373484887,7,Hidden Star (The Star Series),Hidden Star (The Star Series),234.0,['Fiction'],elevenyearold nolityes granny used say mess wo...,Nora Roberts,2001,...,0.571358,0.256229,0.019965,-0.636953,-0.006536,0.099358,153717,"round lake, illinois, usa",27.0,usa
3,62862,0140244824,5,Songs in Ordinary Time (Oprah's Book Club (Pap...,Songs in Ordinary Time (Oprah's Book Club (Pap...,682.0,['Fiction'],new york time bestseller troubled family 1960s...,Mary McGarry Morris,1996,...,0.696558,0.284364,-0.012427,-0.741037,-0.019430,0.027534,62861,"new milford, connecticut, usa",47.0,usa
4,17003,0312305060,10,The Hours: A Novel,The Hours: A Novel,295.0,['Fiction'],michael cunningham brings together pulitzer pr...,Michael Cunningham,2002,...,1.172455,0.515726,-0.061136,-0.634418,0.506026,-0.319519,17002,"carlsbad, california, usa",NaN,usa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93451,251721,1551666766,3,Naughty Marietta,Naughty Marietta,388.0,['Kidnapping victims'],cole heflin agrees find return maxwell laceys ...,Nan Ryan,2003,...,0.526418,0.269839,0.425712,-0.483061,-0.093054,0.309306,251720,"orleans, massachusetts, usa",57.0,usa
93452,275476,0151008116,8,Life of Pi,Life of Pi,352.0,[],one boy one boat one tiger tragic sinking carg...,Yann Martel,2002,...,0.859223,0.280609,0.263873,-0.560046,0.083430,-0.219807,275475,"aspen, colorado, usa",50.0,usa
93453,237898,0375758232,7,Paris to the Moon,Paris to the Moon,368.0,['Biography & Autobiography'],paris name alone conjures image chestnutlined ...,Adam Gopnik,2001,...,0.660590,0.285059,-0.085148,-0.642241,0.067873,0.057663,237897,"salt lake city, utah, usa",NaN,usa
93454,254,0451167317,8,The Dark Half,The Dark Half,496.0,['Fiction'],set fictional town castle rock maine wondrousl...,Stephen King,1994,...,0.851253,0.460726,-0.103199,-0.629326,0.073446,-0.024774,253,"minneapolis, minnesota, usa",24.0,usa


In [ ]:
#Extracting the State information from the location of users
train_features["State"]= ''
for i in range(len(train_features)):
  xxx=(train_features["Location"].iloc[i]).split(",")
  train_features["State"].iloc[i] = xxx[1]


<ipython-input-40-d99d959bac63>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_features["State"].iloc[i] = xxx[1]


In [ ]:
# Filling the empty values of Age with mean
mean_value=train_features['Age'].mean()
train_features["Age"].fillna(value=mean_value, inplace=True)

In [ ]:
# Dropping irrelevant features
train_features=train_features.drop(axis='columns',labels=["Unnamed: 0","Location","country","ISBN","Book-Title","book_name","Synopsis","Book-Author","Publisher"])

In [ ]:
train_features

,User-ID,Book-Rating,page_count,categories,Year-Of-Publication,feature_1,feature_2,feature_3,feature_4,feature_5,...,feature_93,feature_94,feature_95,feature_96,feature_97,feature_98,feature_99,feature_100,Age,State
0,68436,8,706.0,['Fiction'],1996,-0.324090,0.178090,0.085679,0.051699,-0.233513,...,-0.040599,-0.159770,0.468369,0.346564,0.053775,-0.575963,0.202055,0.013086,49.00000,california
1,216829,5,224.0,['Fiction'],1999,-0.265439,0.180581,-0.044998,0.521798,0.064341,...,-0.067668,0.264588,0.671669,0.287058,0.089154,-0.696850,-0.135784,0.029446,37.48389,colorado
2,153718,7,234.0,['Fiction'],2001,-0.353918,0.205880,0.004314,0.376427,-0.038735,...,-0.062447,0.144621,0.571358,0.256229,0.019965,-0.636953,-0.006536,0.099358,27.00000,illinois
3,62862,5,682.0,['Fiction'],1996,-0.372134,0.323817,-0.150502,0.530034,0.088527,...,0.045399,0.196020,0.696558,0.284364,-0.012427,-0.741037,-0.019430,0.027534,47.00000,connecticut
4,17003,10,295.0,['Fiction'],2002,-0.439820,0.646338,0.074526,0.534048,0.210026,...,-0.010307,-0.071196,1.172455,0.515726,-0.061136,-0.634418,0.506026,-0.319519,37.48389,california
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93451,251721,3,388.0,['Kidnapping victims'],2003,-0.431911,0.189106,0.178611,0.353429,0.059231,...,0.055543,0.143617,0.526418,0.269839,0.425712,-0.483061,-0.093054,0.309306,57.00000,massachusetts
93452,275476,8,352.0,[],2002,-0.226763,0.279756,0.136534,0.210636,-0.047499,...,-0.086392,0.063495,0.859223,0.280609,0.263873,-0.560046,0.083430,-0.219807,50.00000,colorado
93453,237898,7,368.0,['Biography & Autobiography'],2001,-0.392849,0.313611,-0.116709,0.383482,-0.036119,...,0.028992,0.084331,0.660590,0.285059,-0.085148,-0.642241,0.067873,0.057663,37.48389,utah
93454,254,8,496.0,['Fiction'],1994,-0.504749,0.368706,0.031612,0.569283,0.101934,...,0.071106,0.127090,0.851253,0.460726,-0.103199,-0.629326,0.073446,-0.024774,24.00000,minnesota


In [ ]:
#Label encoding states and categories
le = LabelEncoder()
train_features['State'] = le.fit_transform(train_features['State'])
train_features['categories'] = le.fit_transform(train_features['categories'])


In [ ]:
train_features

,User-ID,Book-Rating,page_count,categories,Year-Of-Publication,feature_1,feature_2,feature_3,feature_4,feature_5,...,feature_93,feature_94,feature_95,feature_96,feature_97,feature_98,feature_99,feature_100,Age,State
0,68436,8,706.0,330,1996,-0.324090,0.178090,0.085679,0.051699,-0.233513,...,-0.040599,-0.159770,0.468369,0.346564,0.053775,-0.575963,0.202055,0.013086,49.00000,17
1,216829,5,224.0,330,1999,-0.265439,0.180581,-0.044998,0.521798,0.064341,...,-0.067668,0.264588,0.671669,0.287058,0.089154,-0.696850,-0.135784,0.029446,37.48389,23
2,153718,7,234.0,330,2001,-0.353918,0.205880,0.004314,0.376427,-0.038735,...,-0.062447,0.144621,0.571358,0.256229,0.019965,-0.636953,-0.006536,0.099358,27.00000,43
3,62862,5,682.0,330,1996,-0.372134,0.323817,-0.150502,0.530034,0.088527,...,0.045399,0.196020,0.696558,0.284364,-0.012427,-0.741037,-0.019430,0.027534,47.00000,24
4,17003,10,295.0,330,2002,-0.439820,0.646338,0.074526,0.534048,0.210026,...,-0.010307,-0.071196,1.172455,0.515726,-0.061136,-0.634418,0.506026,-0.319519,37.48389,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93451,251721,3,388.0,391,2003,-0.431911,0.189106,0.178611,0.353429,0.059231,...,0.055543,0.143617,0.526418,0.269839,0.425712,-0.483061,-0.093054,0.309306,57.00000,58
93452,275476,8,352.0,500,2002,-0.226763,0.279756,0.136534,0.210636,-0.047499,...,-0.086392,0.063495,0.859223,0.280609,0.263873,-0.560046,0.083430,-0.219807,50.00000,23
93453,237898,7,368.0,127,2001,-0.392849,0.313611,-0.116709,0.383482,-0.036119,...,0.028992,0.084331,0.660590,0.285059,-0.085148,-0.642241,0.067873,0.057663,37.48389,121
93454,254,8,496.0,330,1994,-0.504749,0.368706,0.031612,0.569283,0.101934,...,0.071106,0.127090,0.851253,0.460726,-0.103199,-0.629326,0.073446,-0.024774,24.00000,65


In [ ]:
# extracting query information of users for reranker 
groups = train_features.groupby('User-ID').size().values

In [ ]:
train_features = train_features.drop(axis='columns',labels=["User-ID"])

In [ ]:
train_features

,Book-Rating,page_count,categories,Year-Of-Publication,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,...,feature_93,feature_94,feature_95,feature_96,feature_97,feature_98,feature_99,feature_100,Age,State
0,8,706.0,330,1996,-0.324090,0.178090,0.085679,0.051699,-0.233513,-0.268032,...,-0.040599,-0.159770,0.468369,0.346564,0.053775,-0.575963,0.202055,0.013086,49.00000,17
1,5,224.0,330,1999,-0.265439,0.180581,-0.044998,0.521798,0.064341,-0.506745,...,-0.067668,0.264588,0.671669,0.287058,0.089154,-0.696850,-0.135784,0.029446,37.48389,23
2,7,234.0,330,2001,-0.353918,0.205880,0.004314,0.376427,-0.038735,-0.445708,...,-0.062447,0.144621,0.571358,0.256229,0.019965,-0.636953,-0.006536,0.099358,27.00000,43
3,5,682.0,330,1996,-0.372134,0.323817,-0.150502,0.530034,0.088527,-0.456844,...,0.045399,0.196020,0.696558,0.284364,-0.012427,-0.741037,-0.019430,0.027534,47.00000,24
4,10,295.0,330,2002,-0.439820,0.646338,0.074526,0.534048,0.210026,-0.484646,...,-0.010307,-0.071196,1.172455,0.515726,-0.061136,-0.634418,0.506026,-0.319519,37.48389,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93451,3,388.0,391,2003,-0.431911,0.189106,0.178611,0.353429,0.059231,-0.489475,...,0.055543,0.143617,0.526418,0.269839,0.425712,-0.483061,-0.093054,0.309306,57.00000,58
93452,8,352.0,500,2002,-0.226763,0.279756,0.136534,0.210636,-0.047499,-0.658961,...,-0.086392,0.063495,0.859223,0.280609,0.263873,-0.560046,0.083430,-0.219807,50.00000,23
93453,7,368.0,127,2001,-0.392849,0.313611,-0.116709,0.383482,-0.036119,-0.425903,...,0.028992,0.084331,0.660590,0.285059,-0.085148,-0.642241,0.067873,0.057663,37.48389,121
93454,8,496.0,330,1994,-0.504749,0.368706,0.031612,0.569283,0.101934,-0.427562,...,0.071106,0.127090,0.851253,0.460726,-0.103199,-0.629326,0.073446,-0.024774,24.00000,65


In [ ]:
# Splitting Features and rating into seperate frames
y_train = train_features.iloc[:,1]
x_train = train_features.iloc[:,2:]


In [ ]:
# Converting Data into preferable ranking

ratings = y_train
ratings_scaled = (ratings - 1) / 9

# Convert scaled ratings to ranks
ranks = scipy.stats.rankdata(-ratings_scaled, method='ordinal')

# Update target variable with ranks
Y_train = ranks.astype(int)



## Test Set Features

In [ ]:
# Performing all same process that were done with Training Data 

test_features = test.merge(datas,on="ISBN",how="left")
test_features = test_features.merge(users,on="User-ID",how="left")
test_features["State"]= ''
for i in range(len(test_features)):
  xxx=(test_features["Location"].iloc[i]).split(",")
  test_features["State"].iloc[i] = xxx[1]
mean_value=test_features['Age'].mean()
test_features["Age"].fillna(value=mean_value, inplace=True)
test_features=test_features.drop(axis='columns',labels=["Unnamed: 0","Location","country","ISBN","Book-Title","book_name","Synopsis","Book-Author","Publisher"])
test_features['State'] = le.fit_transform(test_features['State'])
test_features['categories'] = le.fit_transform(test_features['categories'])
test_features = test_features.drop(axis='columns',labels=["User-ID"])
y_test = test_features.iloc[:,1]
x_test = test_features.iloc[:,2:]
ratings = y_test
ratings_scaled = (ratings - 1) / 9

# Convert scaled ratings to ranks
ranks = scipy.stats.rankdata(-ratings_scaled, method='ordinal')

# Update target variable with ranks
Y_test = ranks.astype(int)


## Training the ranker

In [ ]:
from xgboost import XGBRanker
model = XGBRanker(
    objective='rank:ndcg',
    learning_rate=0.05,
    n_estimators=100,
    max_depth=6,
    reg_lambda=1,
    gamma=0,
    min_child_weight=1,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric=['ndcg'],
    verbosity=0,
    random_state=42
)

model.fit(x_train, Y_train, group=groups)

XGBRanker(base_score=None, booster=None, callbacks=None, colsample_bylevel=None,
          colsample_bynode=None, colsample_bytree=0.8,
          early_stopping_rounds=None, enable_categorical=False,
          eval_metric=['ndcg'], feature_types=None, gamma=0, gpu_id=None,
          grow_policy=None, importance_type=None, interaction_constraints=None,
          learning_rate=0.05, max_bin=None, max_cat_threshold=None,
          max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
          max_leaves=None, min_child_weight=1, missing=nan,
          monotone_constraints=None, n_estimators=100, n_jobs=None,
          num_parallel_tree=None, objective='rank:ndcg', predictor=None, ...)

### Hyperparameters information

**objective**='rank:ndcg'  # The ranking objective function to optimize. 'rank:ndcg' optimizes NDCG.

**learning_rate**=0.05  # The learning rate for gradient boosting. Controls the step size during updates.

**n_estimators**=100  # The number of trees in the ensemble.

**max_depth**=6  # The maximum depth of each tree in the ensemble.

**reg_lambda**=1  # L2 regularization term on weights.

**gamma**=0  # Minimum loss reduction required to make a further partition on a leaf node of the tree.

**min_child_weight**=1  # Minimum sum of instance weight needed in a child.

**subsample**=0.8  # Subsample ratio of the training instances.

**colsample_bytree**=0.8  # Subsample ratio of columns when constructing each tree.

**eval_metric**=['ndcg']  # Evaluation metric used to monitor model performance during training. 'ndcg' is the metric used to optimize the objective function.

**verbosity**=0  # Verbosity level of printing messages. 0 means silent.

**random_state**=42  # Seed used by the random number generator.


# Evaluating Model

In [ ]:
def ndcg(y_true, y_pred, k=10):
    ideal_ranking = np.argsort(y_true)[::-1]
    ideal_dcg = np.sum((2**y_true[ideal_ranking[:k]] - 1) / np.log2(np.arange(2, k+2)))
    pred_ranking = np.argsort(y_pred)[::-1]
    pred_dcg = np.sum((2**y_true[pred_ranking[:k]] - 1) / np.log2(np.arange(2, k+2)))
    ndcg = pred_dcg / ideal_dcg
    return ndcg, 

In [ ]:
# Evaluate model on test data
y_pred = model.predict(x_train)
ndcg_scores = ndcg(Y_train, y_pred, k=10)

ndcg_scores

(1.0,)

In [ ]:
# Evaluate model on test data
y_preds= model.predict(x_test)
ndcg_scores1 = ndcg(Y_test, y_preds, k=10)
ndcg_scores1

(1.0,)

# Saving the Reranker Model

In [ ]:
with open('/content/drive/MyDrive/RS Data/Assignment 3/Processed_data/XGB_Ranker.pkl', 'wb') as file:
    pickle.dump(model, file)

In [ ]:
from joblib import dump, load
dump(le, '/content/drive/MyDrive/RS Data/Assignment 3/Processed_data/label_encoder.joblib')

['/content/drive/MyDrive/RS Data/Assignment 3/Processed_data/label_encoder.joblib']

**Note**: LambdaRank and XGBRanker both use gradient boosting to optimize the ranking objective function. The difference between them is the way they calculate the gradient and update the model parameters during training. LambdaRank calculates gradients based on the gradient of NDCG with respect to the predicted scores, while XGBRanker calculates gradients based on pairwise comparisons between documents